In [ ]:
## fully iterative approach

grammar = {
    "S": ["ABC", "X"],
    "A": ["p", "^"],
    "B": ["q", "^"],
    "C": ["(S)", "^"],
    "X": ["^C"]
}

def first_of(symbol):
    first_set = set()
    stack = [symbol]
    visited = set()

    while stack:
        current = stack.pop()
        if current in visited:
            continue
        visited.add(current)

        if current not in grammar:
            first_set.add(current)  # Terminal symbol
        else:
            for production in grammar[current]:
                i = 0
                while i < len(production):
                    char = production[i]
                    if char not in grammar:  # Terminal
                        first_set.add(char)
                        break
                    else:
                        nullable = False
                        stack.append(char)

                        # Check if FIRST(char) contains epsilon ('^') without recursion
                        for sub_production in grammar[char]:
                            if "^" in sub_production:  # If any production of 'char' has '^', it can be nullable
                                nullable = True
                                break  # No need to check further, we found '^'

                        if not nullable:  # If the non-terminal is not nullable, stop processing further symbols
                            break
                    i += 1

    return first_set

# Example usage:
print(first_of("X"))  # Expected output: {'p', 'q', '(', '^'}

{'^'}


In [1]:
## iteration for checking NT contain null or not replaced by recursive upproach.
grammar = {
    "S": ["ABC", "X"],
    "A": ["p", "^"],
    "B": ["q", "^"],
    "C": ["(S)", "^"],
    "X": ["^C"]
}

def first_of(symbol):
    first_set = set()
    stack = [symbol]
    visited = set()

    while stack:
        current = stack.pop()
        if current in visited:
            continue
        visited.add(current)

        if current not in grammar:
            first_set.add(current)  # Terminal symbol
        else:
            for production in grammar[current]:
                i = 0
                while i < len(production):
                    char = production[i]
                    if char not in grammar:  # Terminal
                        first_set.add(char)
                        break
                    else:
                        stack.append(char)
                        if "^" not in first_of(char):  # Stop if non-terminal does not have epsilon
                            break
                    i += 1
                else:  # If all symbols in production are nullable
                    first_set.add("^")

    return first_set

# Example usage:
print(first_of("S"))  # Expected output: {'p', 'q', '(', '^'}


{'^', 'q', 'p', '('}


In [7]:
def follow_of(symbol):
    follow_set = {symbol: set() for symbol in grammar}
    follow_set["S"].add("$")  # Start symbol gets EOF

    while True:
        updated = False
        for head, productions in grammar.items():
            for production in productions:
                trailer = follow_set[head].copy()
                for i in range(len(production) - 1, -1, -1):
                    char = production[i]
                    if char in grammar:  # Non-terminal
                        if follow_set[char] | trailer != follow_set[char]:
                            follow_set[char] |= trailer
                            updated = True
                        if "^" in first_of(char):
                            trailer |= first_of(char) - {"^"}
                        else:
                            trailer = first_of(char)
                    else:
                        trailer = {char}
        if not updated:
            break

    return follow_set[symbol]

# Example usage:
print("FIRST(X):", first_of("X"))  # Expected output: {'p', 'q', '(', '^'}
print("FOLLOW(X):", follow_of("X"))

FIRST(X): {'^'}
FOLLOW(X): {')', '$'}
